In [1]:
import pandas as pd
import requests, zipfile, io
from datetime import timedelta, datetime
import numpy as np

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


N_DAYS_AGO = 6
today = datetime.now()    
n_days_ago = today - timedelta(days=N_DAYS_AGO)

today = today.strftime("%Y%m%d")
prior = n_days_ago.strftime("%Y%m%d")


# zip_old = requests.get('https://www.grants.gov/extract/GrantsDBExtract' + prior + 'v2.zip')
# zip_new = requests.get('https://www.grants.gov/extract/GrantsDBExtract' + today + 'v2.zip')


# o = zipfile.ZipFile(io.BytesIO(zip_old.content))
# o.extractall()

# n = zipfile.ZipFile(io.BytesIO(zip_new.content))
# n.extractall()


df_old_orig = pd.read_xml("./GrantsDBExtract" + prior + "v2.xml")
df_new_orig = pd.read_xml("./GrantsDBExtract" + today + "v2.xml")


identifying_column = "OpportunityID"
name_column = "OpportunityTitle"
cfda_num_list = [98, 19, 10]
agency_list = ["USAID", "USDS", "USDA"]
message = ""

df_dict = {}

for agency, cfda_num in zip(agency_list, cfda_num_list):
    
    def see_if_nan (x):
        try:
            return int(x)
        except:
            return 0
        
    df_new = df_new_orig
    df_old = df_old_orig

    cfda_categories_old = [see_if_nan(x) for x in df_old['CFDANumbers'].values.tolist()]
    cfda_categories_new = [see_if_nan(x) for x in df_new['CFDANumbers'].values.tolist()]

    df_new['CFDACategory'] = cfda_categories_new
    df_old['CFDACategory'] = cfda_categories_old

    df_new = df_new[(df_new["CFDACategory"].isin([cfda_num]))]
    df_old = df_old[(df_old["CFDACategory"].isin([cfda_num]))]

    
    old_list = df_old[identifying_column].values.tolist()
    new_list = df_new[identifying_column].values.tolist()

    
    #in old but not in new -- removed
    removed = list(set(old_list) - set(new_list))
    print(removed)

    
    #in new but not in old -- added
    added = list(set(new_list) - set(old_list))
    print(added)

    
    removed_df = df_old[df_old[identifying_column].isin(removed)]
    df_old_final = df_old[~df_old[identifying_column].isin(removed)]

    
    added_df = df_new[df_new[identifying_column].isin(added)]
    df_new_final = df_new[~df_new[identifying_column].isin(added)]

    
    # df_new = df_new.reindex(columns=df_old.columns)

    
    df_new_final = df_new_final.sort_values(by=[identifying_column])
    df_old_final = df_old_final.sort_values(by=[identifying_column])

    
    df_new_final = df_new_final.reset_index(drop=True)
    df_old_final = df_old_final.reset_index(drop=True)

    
    df_new_final.compare(df_old_final)

    
    comparison_df = df_new_final.compare(df_old_final)

    df_dict[agency] = [added_df, removed_df, comparison_df, df_new_final, df_old_final]

[]
[350711, 350639]
[350178]
[350706, 350634, 350693, 350623]
[]
[350641, 350698]


In [10]:
def refine_df(df, countries):
    if not df.empty:
        remove_nones = ["Description", "AgencyName", "AdditionalInformationOnEligibility"]
        for column in remove_nones:
            blank_list = []
            for i in df[column]:
                if type(i) == str:
                    blank_list.append(i)
                else:
                    blank_list.append("No " + column)
            df[column] = blank_list

        df_copy = df[
            ~df["OpportunityTitle"].str.contains('|'.join(countries))
            & ~df["AgencyName"].str.contains('|'.join(countries)) 
            & ~df["AdditionalInformationOnEligibility"].str.contains('|'.join(countries))
            & ~df["Description"].str.contains('|'.join(countries))]
    else:
        df_copy = df

    return df_copy

In [31]:
usaid_added = df_dict['USAID'][0].head()
usaid_removed = df_dict['USAID'][1].head()
usaid_df_new_final = df_dict['USAID'][3].head()
usaid_df_old_final = df_dict['USAID'][4].head()


In [33]:
usaid_df_new_final.tail()

,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,...,OpportunityCategoryExplanation,EstimatedSynopsisPostDate,FiscalYear,EstimatedSynopsisCloseDate,EstimatedSynopsisCloseDateExplanation,EstimatedAwardDate,EstimatedProjectStartDate,GrantorContactName,GrantorContactPhoneNumber,CFDACategory
0,40034,Human Rights Technical Assistance in Ethiopia,663-A-08-002,D,CA,O,"supporting the establishment of a vibrant, mul...",98.001,25.0,All qualified applicants,...,None,NaN,NaN,NaN,None,NaN,NaN,None,None,98
1,40111,Guidelines and Procedures for Fiscal Year 2008...,001,D,G,HL,None,98.007,25.0,"To be eligible for a grant, an organization mu...",...,None,NaN,NaN,NaN,None,NaN,NaN,None,None,98
2,40116,Transforming Sierra Leone Draft Program Descri...,TBD636,D,CA,NR,None,98.001,99.0,None,...,None,NaN,NaN,NaN,None,NaN,NaN,None,None,98
3,40156,Public-Private Alliances,USAID-M-OAA-08-108-APS,D,O,IS,None,98.011,99.0,None,...,None,NaN,NaN,NaN,None,NaN,NaN,None,None,98
4,40157,Care Services for HIV - Infected Orphans and V...,APS-663-08-004,D,O,HL,None,98.001,99.0,None,...,None,NaN,NaN,NaN,None,NaN,NaN,None,None,98


In [5]:
countries = open("countries_fh.txt", "r")
countries = countries.read()
countries = countries.split("\n")

In [25]:
usaid_added = refine_df(usaid_added, countries)
usaid_removed = refine_df(usaid_removed, countries)
usaid_df_new_final = refine_df(usaid_df_new_final, countries)
usaid_df_old_final = refine_df(usaid_df_old_final, countries)

C:\Users\nhagl\AppData\Local\Temp\ipykernel_29844\148130369.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = blank_list


In [26]:
usaid_df_new_final
comparison_df = usaid_df_new_final.compare(usaid_df_old_final)

In [19]:
# usaid_df_old_final_copy = usaid_df_old_final[
#     ~usaid_df_old_final["OpportunityTitle"].str.contains('|'.join(countries))
#     & ~usaid_df_old_final["AgencyName"].str.contains('|'.join(countries)) 
#     & ~usaid_df_old_final["AdditionalInformationOnEligibility"].str.contains('|'.join(countries))
#     & ~usaid_df_old_final["Description"].str.contains('|'.join(countries))]

# usaid_df_new_final_copy = usaid_df_new_final[
#     ~usaid_df_new_final["OpportunityTitle"].str.contains('|'.join(countries))
#     & ~usaid_df_new_final["AgencyName"].str.contains('|'.join(countries)) 
#     & ~usaid_df_new_final["AdditionalInformationOnEligibility"].str.contains('|'.join(countries))
#     & ~usaid_df_new_final["Description"].str.contains('|'.join(countries))]

In [41]:
# for i in countries:
#     if list(added_usaid_copy['Description'].str.contains(i))[0]:
#         print(i)

United States
